In [45]:
import glob,sys
from pathlib import Path
pinkRig_path= glob.glob(r'C:\Users\*\Documents\Github\PinkRigs')
pinkRig_path = Path(pinkRig_path[0])
sys.path.insert(0, (pinkRig_path.__str__()))
kilofolder = __import__("Analysis.+kilo.python_.convert_to_ibl_format") # this is necessary because some wise people decided to write a package with python and matlab in it. #wisdom.
kilo = getattr(kilofolder, "+kilo")
add_anat_to_ibl_format = kilo.python_.convert_to_ibl_format.add_anat_to_ibl_format

# Path to raw ephys data
#ephys_path = Path(r'Z:\AV014\2022-07-07\ephys\AV014_2022-07-07_NatImagesSparseNoise_g0\AV014_2022-07-07_NatImagesSparseNoise_g0_imec0')
     
#add_anat_to_ibl_format(ephys_path,ks_folder='pyKS',recompute=True)

In [ ]:
# add a func to loop over SC recordings (selected somehow by Flora, what a shame we cannot pay for someone competent.)

import pandas as pd 
from Admin.csv_pyhandlers import get_server_location
from Analysis.helpers.queryExp import load_data

prPath = get_server_location()
session_list = pd.read_csv(prPath / 'Helpers/sc_selected_recordings.csv')


for _,r in session_list.iterrows():
    recdat = load_data(subject = r.Subject,expDate=r.expDate, expNum = r.expNum)
    # now I will try to convert all files that were already converted to ibl format
    ephys_path = Path(recdat.iloc[0].expFolder).parent / 'ephys'
    ephys_recordings = list(ephys_path.glob('**\pyKS\output\ibl_format\channels.localCoordinates.npy'))

    for rec in ephys_recordings: 
            s=add_anat_to_ibl_format(rec.parents[3],recompute= True)
            print(rec)